In [1]:
from QHyper.solvers.quantum_annealing.advantage import Advantage

import numpy as np
from collections import defaultdict

from QHyper.converter import Converter
from QHyper.constraint import Polynomial

from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel
from dimod.sampleset import SampleSet

In [2]:
def convert_qubo_keys(qubo: Polynomial) -> tuple[dict[tuple, float], float]:
    new_qubo = defaultdict(float)
    offset = 0.0

    qubo, offset = qubo.separate_const()
    for k, v in qubo.terms.items():
        if len(k) == 1:
            new_key = (k[0], k[0])
        elif len(k) > 2:
            raise ValueError("Only supports quadratic model")
        else:
            new_key = k

        new_qubo[new_key] += v

    return (new_qubo, offset)

In [3]:
import networkx as nx


Graphs = np.load("networks/powerlaw_m=1_p=0.2/graphs.npy", allow_pickle=True)
G = Graphs[9]

In [4]:
from QHyper.problems.community_detection import CommunityDetectionProblem, Network


network = Network(graph=G)
problem = CommunityDetectionProblem(network_data=network, communities=2, one_hot_encoding=False)

In [5]:
sampler = DWaveSampler(solver="Advantage_system5.4", region="eu-central-1")
embedding_compose = EmbeddingComposite(sampler)

In [7]:
qubo = Converter.create_qubo(problem, [])
qubo_terms, offset = convert_qubo_keys(qubo)
bqm = BinaryQuadraticModel.from_qubo(qubo_terms, offset=offset)

In [38]:
sampleset = EmbeddingComposite(sampler).sample(
    bqm, num_reads=100
)

In [39]:
res = sampleset.first.sample

In [40]:
c0 = sorted([int(k[1:]) for k, v in res.items() if v == 0])
c1 = sorted([int(k[1:]) for k, v in res.items() if v == 1])

print(len(c0))
print(len(c1))
print(nx.community.modularity(G, [c0, c1]))

46
54
0.4421487603305785


In [12]:
sampleset = embedding_compose.sample(
    bqm, num_reads=100, return_embedding=True
)

sampleset

In [14]:
embedding_from_sampleset = sampleset.info["embedding_context"]["embedding"]

In [8]:
import minorminer
import dimod


find_embedding=minorminer.find_embedding
child_structure_search = dimod.child_structure_dfs

In [9]:
# apply the embedding to the given problem to map it to the child sampler
target_structure = child_structure_search(sampler)
__, target_edgelist, target_adjacency = target_structure

# add self-loops to edgelist to handle singleton variables
source_edgelist = list(bqm.quadratic) + [(v, v) for v in bqm.linear]

In [10]:
embedding = find_embedding(source_edgelist, target_edgelist)

In [15]:
embedding_calculated = embedding

In [19]:
embedding_calculated["x3"]

[695, 696, 697, 3888, 3887, 395, 693, 3931, 3930, 694, 692, 691, 698, 690]

In [20]:
embedding_from_sampleset["x3"]

(797, 798, 799, 800, 796, 795, 3107, 3108, 801, 802, 3106)

In [21]:
sampleset_with_random_seed = embedding_compose.sample(
    bqm, num_reads=100, return_embedding=True, embedding_parameters={"random_seed": 10}
)

In [22]:
embedding_with_random_seed = find_embedding(source_edgelist, target_edgelist, random_seed=10)

In [23]:
embedding_from_sampleset_rs = sampleset_with_random_seed.info["embedding_context"]["embedding"]

In [24]:
embedding_calculated_rs = embedding_with_random_seed

In [30]:
sorted(embedding_calculated_rs) == sorted(embedding_from_sampleset_rs)

True

In [32]:
from dwave.system.composites import FixedEmbeddingComposite


embedding_compose_fe = FixedEmbeddingComposite(sampler, embedding=embedding_calculated_rs)

In [33]:
sampleset_fe = embedding_compose_fe.sample(bqm, num_reads=100, return_embedding=True)

In [36]:
res_fe = sampleset_fe.first.sample

In [41]:
c0_fe = sorted([int(k[1:]) for k, v in res_fe.items() if v == 0])
c1_fe = sorted([int(k[1:]) for k, v in res_fe.items() if v == 1])

In [50]:
print(nx.community.modularity(G, [c0, c1]))
print(nx.community.modularity(G, [c0_fe, c1_fe]))

0.4421487603305785
0.44485256606468726


In [52]:
from joblib import Parallel, delayed


N_RUNS = 8
n_jobs = 8

embeddings = Parallel(n_jobs=n_jobs)(
                    delayed(find_embedding)(source_edgelist, target_edgelist, random_seed=10) for _ in range(N_RUNS)
                )

In [55]:
embeddings_2 = []

for _ in range(N_RUNS):
    emb = find_embedding(source_edgelist, target_edgelist, random_seed=10)
    embeddings_2.append(emb)

In [60]:
problem_c0 = CommunityDetectionProblem(network_data=Network(graph=G, community=c0), communities=2, one_hot_encoding=False)
qubo_c0 = Converter.create_qubo(problem_c0, [])
qubo_terms_c0, offset_c0 = convert_qubo_keys(qubo_c0)
bqm_c0 = BinaryQuadraticModel.from_qubo(qubo_terms_c0, offset=offset_c0)

In [61]:
source_edgelist_c0 = list(bqm_c0.quadratic) + [(v, v) for v in bqm_c0.linear]
embeddings_c0 = Parallel(n_jobs=n_jobs)(
                    delayed(find_embedding)(source_edgelist_c0, target_edgelist, random_seed=10) for _ in range(N_RUNS)
                )

In [65]:
embeddings_2_c0 = []

for _ in range(N_RUNS):
    emb = find_embedding(source_edgelist_c0, target_edgelist, random_seed=10)
    embeddings_2_c0.append(emb)

In [66]:
def run_prof():
    s = FixedEmbeddingComposite(sampler, embedding=embedding_calculated_rs).sample(bqm, num_reads=100)
    res = s.first.sample
    return res

In [73]:
for i in range(10):
    single = run_prof()
    print(single)

{'x0': 1, 'x1': 1, 'x10': 1, 'x11': 0, 'x12': 1, 'x13': 1, 'x14': 1, 'x15': 0, 'x16': 0, 'x17': 1, 'x18': 0, 'x19': 0, 'x2': 1, 'x20': 1, 'x21': 1, 'x22': 0, 'x23': 0, 'x24': 1, 'x25': 0, 'x26': 0, 'x27': 0, 'x28': 1, 'x29': 1, 'x3': 1, 'x30': 1, 'x31': 1, 'x32': 0, 'x33': 1, 'x34': 0, 'x35': 1, 'x36': 1, 'x37': 1, 'x38': 1, 'x39': 1, 'x4': 0, 'x40': 0, 'x41': 1, 'x42': 1, 'x43': 0, 'x44': 0, 'x45': 1, 'x46': 0, 'x47': 0, 'x48': 1, 'x49': 0, 'x5': 1, 'x50': 1, 'x51': 0, 'x52': 0, 'x53': 0, 'x54': 0, 'x55': 0, 'x56': 1, 'x57': 1, 'x58': 0, 'x59': 0, 'x6': 1, 'x60': 0, 'x61': 1, 'x62': 0, 'x63': 0, 'x64': 0, 'x65': 1, 'x66': 1, 'x67': 1, 'x68': 1, 'x69': 0, 'x7': 0, 'x70': 1, 'x71': 0, 'x72': 0, 'x73': 0, 'x74': 0, 'x75': 1, 'x76': 0, 'x77': 0, 'x78': 1, 'x79': 0, 'x8': 1, 'x80': 1, 'x81': 0, 'x82': 0, 'x83': 0, 'x84': 0, 'x85': 1, 'x86': 1, 'x87': 1, 'x88': 1, 'x89': 0, 'x9': 0, 'x90': 0, 'x91': 0, 'x92': 0, 'x93': 0, 'x94': 1, 'x95': 1, 'x96': 1, 'x97': 1, 'x98': 0, 'x99': 1}
{'x0': 1,

In [79]:
def run_with_find_emb():
    emb = find_embedding(source_edgelist, target_edgelist)
    s = FixedEmbeddingComposite(sampler, embedding=emb).sample(bqm, num_reads=100)
    return s.first.sample

In [ ]:
%timeit run_with_find_emb()

In [82]:
def classical():    
    res = embedding_compose.sample(bqm, num_reads=100)
    return res.first.sample

In [83]:
%time classical()

CPU times: total: 56.1 s
Wall time: 1min 9s


{'x0': 1,
 'x1': 0,
 'x10': 1,
 'x11': 1,
 'x12': 1,
 'x13': 0,
 'x14': 1,
 'x15': 0,
 'x16': 0,
 'x17': 1,
 'x18': 0,
 'x19': 0,
 'x2': 1,
 'x20': 1,
 'x21': 1,
 'x22': 1,
 'x23': 0,
 'x24': 1,
 'x25': 0,
 'x26': 0,
 'x27': 0,
 'x28': 0,
 'x29': 1,
 'x3': 1,
 'x30': 1,
 'x31': 1,
 'x32': 1,
 'x33': 0,
 'x34': 0,
 'x35': 0,
 'x36': 1,
 'x37': 1,
 'x38': 1,
 'x39': 1,
 'x4': 0,
 'x40': 0,
 'x41': 0,
 'x42': 1,
 'x43': 0,
 'x44': 0,
 'x45': 1,
 'x46': 0,
 'x47': 0,
 'x48': 1,
 'x49': 0,
 'x5': 1,
 'x50': 1,
 'x51': 0,
 'x52': 0,
 'x53': 0,
 'x54': 0,
 'x55': 1,
 'x56': 1,
 'x57': 1,
 'x58': 0,
 'x59': 0,
 'x6': 1,
 'x60': 1,
 'x61': 1,
 'x62': 1,
 'x63': 1,
 'x64': 1,
 'x65': 1,
 'x66': 1,
 'x67': 1,
 'x68': 1,
 'x69': 0,
 'x7': 1,
 'x70': 1,
 'x71': 0,
 'x72': 0,
 'x73': 1,
 'x74': 0,
 'x75': 1,
 'x76': 0,
 'x77': 0,
 'x78': 1,
 'x79': 1,
 'x8': 1,
 'x80': 1,
 'x81': 0,
 'x82': 0,
 'x83': 0,
 'x84': 1,
 'x85': 1,
 'x86': 1,
 'x87': 1,
 'x88': 1,
 'x89': 0,
 'x9': 0,
 'x90': 0,
 'x91': 0

In [68]:
%load_ext line_profiler

In [70]:
%lprun -f FixedEmbeddingComposite.sample run_prof()

Timer unit: 1e-07 s

Total time: 0.293546 s
File: c:\Users\basia\AppData\Local\Programs\Python\Python310\lib\site-packages\dwave\system\composites\embedding.py
Function: sample at line 437

Line #      Hits         Time  Per Hit   % Time  Line Contents
   437                                               def sample(self, bqm, **parameters):
   438                                                   """Sample the binary quadratic model.
   439                                           
   440                                                   On the first call of a sampling method, finds a :term:`minor-embedding`
   441                                                   for the given binary quadratic model (BQM). All subsequent calls to its
   442                                                   sampling methods reuse this embedding.
   443                                           
   444                                                   Args:
   445                                       

In [72]:
%prun -s cumulative run_prof()

         318594 function calls (307294 primitive calls) in 0.937 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.941    0.941 {built-in method builtins.exec}
        1    0.000    0.000    0.940    0.940 2325209082.py:1(run_prof)
        1    0.000    0.000    0.747    0.747 sampleset.py:1065(first)
    18/15    0.000    0.000    0.747    0.050 {built-in method builtins.next}
        2    0.000    0.000    0.747    0.373 sampleset.py:1220(data)
      8/1    0.000    0.000    0.747    0.747 sampleset.py:1105(record)
     22/2    0.000    0.000    0.747    0.373 sampleset.py:1480(resolve)
        1    0.000    0.000    0.747    0.747 embedding.py:279(async_unembed)
        1    0.000    0.000    0.746    0.746 transforms.py:513(unembed_sampleset)
        1    0.000    0.000    0.741    0.741 dwave_sampler.py:426(_hook)
        1    0.000    0.000    0.741    0.741 dwave_sampler.py:427(resolv